# Data Utilities

Some python scripts for munging dictionaries and other data tasks.

In [6]:
# Global imports

import re
import json
import csv

In [7]:
DATA_DIR = '../data';

alpha_only = re.compile(r'[a-zA-Z]+')

def get_words(dict_file_name):
  with open(f"{DATA_DIR}/{dict_file_name}", encoding='latin-1', mode='r') as f:
    words = f.read().splitlines()

  print(f"{len(words):,} words read from {dict_file_name}")

  words = [word.lower() for word in words if alpha_only.match(word)]
  print(f"... {len(words):,} are alpha only")

  words = [word for word in words if len(word) == 5]
  print(f"... {len(words):,} are 5 letters long\n")

  return set(words)

In [5]:
dict = get_words('dict.txt')
usa = get_words('usa2.txt')
english = get_words('english3.txt')
wordle_solutions = get_words('wordle_solutions.txt')
wordle_dict = get_words('wordle_complete_dictionary.txt')

# Confirm all solutions are IN the complete dictionary!
present = wordle_solutions.intersection(wordle_dict)
print(f"{len(present):,} solutions are in the complete dictionary")


42,700 words read from dict.txt
... 42,697 are alpha only
... 4,430 are 5 letters long

77,722 words read from usa2.txt
... 77,718 are alpha only
... 5,442 are 5 letters long

194,433 words read from english3.txt
... 194,433 are alpha only
... 11,435 are 5 letters long

2,315 words read from wordle_solutions.txt
... 2,315 are alpha only
... 2,315 are 5 letters long

12,897 words read from wordle_complete_dictionary.txt
... 12,897 are alpha only
... 12,897 are 5 letters long

2,315 solutions are in the complete dictionary


In [29]:
# My dictionary missing many words!
# isect = dict.intersection(usa)
isect = usa.intersection(english)
print(f"{len(isect):,} words in usa ISECT english")

5,282 words in uses ISECT english


In [14]:
# Use the same dictionaries as used in Wordle.

with open(f'{DATA_DIR}/words.json', 'w') as f:
  json.dump(sorted(wordle_dict), f, indent=2)

with open(f'{DATA_DIR}/solutions.json', 'w') as f:
  json.dump(sorted(wordle_solutions), f, indent=2)

In [30]:
# Write a json file with this cleanup up dictionary
# import json

# with open(f'{DATA_DIR}/words.json', 'w') as f:
#   json.dump(sorted(isect), f, indent=2)


# Test Words

In [9]:
words = []

with open(f"{DATA_DIR}/darrellp.csv", 'r') as f:
  reader = csv.reader(f)
  words = [row[0] for row in reader]

print(f"{len(words):,} words read from darrellp.csv")

with open(f"{DATA_DIR}/test-words.json", 'w') as f:
  json.dump(words, f, indent=2)

722 words read from darrellp.csv


# Evaluate Test Results

In [11]:
def analyze_results(file_name):
  with open(f"{DATA_DIR}/{file_name}", 'r') as f:
    reader = csv.reader(f)
    scores = [row[2] for row in reader]

  samples = len(scores)

  num_scores = [int(score) for score in scores if score.isnumeric()]

  print(f"{file_name} average: {sum(num_scores) / len(num_scores):.2f} over " +
    f"{len(num_scores):,} scores")

In [15]:
analyze_results('darrellp.csv')
analyze_results('mckoss.csv')

darrellp.csv average: 4.18 over 722 scores
mckoss.csv average: 3.63 over 365 scores
